<a href="https://colab.research.google.com/github/shivi13102/FinMitra/blob/main/AI_Investment_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from cryptography.fernet import Fernet
import os

from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

In [2]:
rural_user_data = pd.read_csv('rural_indian_user_profiles.csv')

In [3]:
rural_user_data.columns

Index(['Full Name', 'Age', 'State', 'Village', 'Occupation', 'Annual Income',
       'Monthly Expenses', 'Net Worth', 'Existing Investments',
       'Risk Tolerance', 'Investment Time Horizon',
       'Preferred Investment Types', 'Financial Goals', 'Goal Amount',
       'Goal Timeline', 'Savings Behavior', 'Spending Habits',
       'Emergency Fund Status', 'PAN Card Number', 'Bank Account Details',
       'Aadhaar Number', 'Phone Number'],
      dtype='object')

In [4]:
rural_user_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300 entries, 0 to 299
Data columns (total 22 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   Full Name                   300 non-null    object
 1   Age                         300 non-null    int64 
 2   State                       300 non-null    object
 3   Village                     300 non-null    object
 4   Occupation                  300 non-null    object
 5   Annual Income               300 non-null    int64 
 6   Monthly Expenses            300 non-null    int64 
 7   Net Worth                   300 non-null    int64 
 8   Existing Investments        231 non-null    object
 9   Risk Tolerance              300 non-null    object
 10  Investment Time Horizon     300 non-null    object
 11  Preferred Investment Types  300 non-null    object
 12  Financial Goals             300 non-null    object
 13  Goal Amount                 300 non-null    int64 

In [5]:
display(rural_user_data)

,Full Name,Age,State,Village,Occupation,Annual Income,Monthly Expenses,Net Worth,Existing Investments,Risk Tolerance,...,Financial Goals,Goal Amount,Goal Timeline,Savings Behavior,Spending Habits,Emergency Fund Status,PAN Card Number,Bank Account Details,Aadhaar Number,Phone Number
0,उमा नाथ,46,Madhya Pradesh,Akbarpur,Grocer,191780,17818,553797,Bonds,Low,...,Small Business,204220,5,11,Discretionary,No,3041200930,GB49ENME76647119379640,755266556380,09537 672264
1,दीप्ति पटेल,39,West Bengal,Dungarpur,Other,377618,23446,976583,Real Estate,Moderate,...,Emergency Fund,396643,10,23,Essential,Yes,2414164990,GB03KMDL75514382081345,643625695673,004-23752614
2,देवी मेहता,21,Bihar,Koriya,Teacher,236866,14944,854051,NaN,Low,...,Agriculture Improvement,459938,1,21,Discretionary,Yes,7299724579,GB84LVOB37910603127969,794301329020,09751 028486
3,प्रतिमा द्विवेदी,49,Rajasthan,Chanchal,Other,267227,21228,750495,Real Estate,High,...,Education,250307,3,18,Essential,Yes,2328120286,GB45STZE65543827651768,212142531566,+91 6007385673
4,लीना घोष,24,West Bengal,Chaibasa,Farmer,488581,23714,329614,Real Estate,Low,...,Education,497144,3,23,Discretionary,Yes,7450467385,GB50DTPC50717634216074,861570408223,09650137495
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,इन्द्रजित रोद्रिगुएस,44,Rajasthan,Sheopur,Farmer,362880,24692,836410,Mutual Funds,Moderate,...,Retirement,328378,1,14,Essential,No,4595592204,GB74EIDN11536674861023,983362094925,+91 452 8710589
296,लक्ष्मी बलासुब्रमानियम,28,Odisha,Ratangarh,Agriculture Laborer,583421,18424,758251,Real Estate,High,...,Education,256463,3,19,Essential,Yes,8988291769,GB43DEUL82892834807041,301499796048,+91 461 7941466
297,शीला वर्मा,33,Odisha,Kendrapara,Small Business Owner,253688,25982,67633,Mutual Funds,Low,...,Agriculture Improvement,471260,10,17,Discretionary,No,4751366283,GB06LWPT10058872269178,782839032489,+91 883 1765233
298,माधवी गुहा,42,Jharkhand,Sheopur,Shopkeeper,191426,29750,428037,NaN,Low,...,Education,119921,5,15,Discretionary,Yes,8414195092,GB39DCEH83542925508652,816605597171,+91 1394222653


# Remove PII for Privacy Compliance:

Mask or encrypt sensitive information like *PAN Card Number, Bank Account Details, Aadhaar Number, and Phone Number*.

In [11]:
# File to store the encryption key
key_file_path = "encryption_key.key"

# Generate a new encryption key if it doesn't already exist
if not os.path.exists(key_file_path):
    key = Fernet.generate_key()
    with open(key_file_path, "wb") as key_file:
        key_file.write(key)
    print("Encryption key generated and saved.")
else:
    # Load the existing encryption key
    with open(key_file_path, "rb") as key_file:
        key = key_file.read()
    print("Encryption key loaded.")

cipher = Fernet(key)

def mask_phone(phone):
    """Mask phone numbers to keep only the last 4 digits."""
    phone_str = str(phone)
    return "XXXXXX" + phone_str[-4:]

def encrypt_data(data):
    """Encrypt sensitive data using Fernet encryption."""
    data_str = str(data)  # Ensure the data is a string
    return cipher.encrypt(data_str.encode()).decode()

# Load the dataset
file_path = "rural_indian_user_profiles.csv"
df = pd.read_csv(file_path)

# Mask or encrypt sensitive columns
df['PAN Card Number'] = df['PAN Card Number'].apply(encrypt_data)
df['Bank Account Details'] = df['Bank Account Details'].apply(encrypt_data)
df['Aadhaar Number'] = df['Aadhaar Number'].apply(encrypt_data)
df['Phone Number'] = df['Phone Number'].apply(mask_phone)

# Save the processed dataset to a new file
output_path = "processed_user_profiles.csv"
df.to_csv(output_path, index=False)

print(f"Processed data saved to {output_path}")

Encryption key loaded.
Processed data saved to processed_user_profiles.csv


In [12]:
df= pd.read_csv('/content/processed_user_profiles.csv')
display(df)

,Full Name,Age,State,Village,Occupation,Annual Income,Monthly Expenses,Net Worth,Existing Investments,Risk Tolerance,...,Financial Goals,Goal Amount,Goal Timeline,Savings Behavior,Spending Habits,Emergency Fund Status,PAN Card Number,Bank Account Details,Aadhaar Number,Phone Number
0,उमा नाथ,46,Madhya Pradesh,Akbarpur,Grocer,191780,17818,553797,Bonds,Low,...,Small Business,204220,5,11,Discretionary,No,gAAAAABnf1FDxIQlk4ZCg-kYt5027CLqV_AjY7WElyO90T...,gAAAAABnf1FDCXVkwqPfH59VF8IRS-h_zg_zoAp9DekZHu...,gAAAAABnf1FDdTDbzIuo5cZPYUm7RiC5oLU0-iEKdl5gci...,XXXXXX2264
1,दीप्ति पटेल,39,West Bengal,Dungarpur,Other,377618,23446,976583,Real Estate,Moderate,...,Emergency Fund,396643,10,23,Essential,Yes,gAAAAABnf1FDu55pK6ok8n7K_SzUfT_gIBflpRqXd5VtVq...,gAAAAABnf1FDa122MRKchEYKUoc3Ld4m_Hwy4_P-MpoM-u...,gAAAAABnf1FDkOy9lbQWJ4eJmzVceP7CSjyshVAlDtcsE8...,XXXXXX2614
2,देवी मेहता,21,Bihar,Koriya,Teacher,236866,14944,854051,NaN,Low,...,Agriculture Improvement,459938,1,21,Discretionary,Yes,gAAAAABnf1FDW1UF1i9MCt96Sqg01DW1JMHbpxukOY9V1x...,gAAAAABnf1FDQi5nFBmxekGaqx6jChiIHImy3ys3D9bzAv...,gAAAAABnf1FDmqoiDblms00Tjh-EjGZdH4ezyniKXTj5-8...,XXXXXX8486
3,प्रतिमा द्विवेदी,49,Rajasthan,Chanchal,Other,267227,21228,750495,Real Estate,High,...,Education,250307,3,18,Essential,Yes,gAAAAABnf1FDNHPetVHJhw50uga_rm_CCbNzElmZ9D-R7D...,gAAAAABnf1FDJJt7irV8r5kiy7GiLUdrlHYrt_3_wSIy8e...,gAAAAABnf1FDYebFBLc2J-Mk2QhcE0wBAMEkjSN-5ZTkVC...,XXXXXX5673
4,लीना घोष,24,West Bengal,Chaibasa,Farmer,488581,23714,329614,Real Estate,Low,...,Education,497144,3,23,Discretionary,Yes,gAAAAABnf1FD04hthv9SLqgn1Rwre4sO4_Ro4MdTJDGzrJ...,gAAAAABnf1FDFGtrk6dgL7qyiAbhvd12BaFZs0LGCX3_eU...,gAAAAABnf1FDQ_5hDGqzRz-dv-AmfPUZXeGzmtr6YjhZAl...,XXXXXX7495
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,इन्द्रजित रोद्रिगुएस,44,Rajasthan,Sheopur,Farmer,362880,24692,836410,Mutual Funds,Moderate,...,Retirement,328378,1,14,Essential,No,gAAAAABnf1FDhKFX9nagxxFQpGTJED8hbSI1Kv_yVOpXa7...,gAAAAABnf1FDOnxUkrNJI1nGtb-oSSWu_KhUdTqotz4njG...,gAAAAABnf1FDXLQ1X58seKyk-lUWKD0tEIiJTrFbYnKxgO...,XXXXXX0589
296,लक्ष्मी बलासुब्रमानियम,28,Odisha,Ratangarh,Agriculture Laborer,583421,18424,758251,Real Estate,High,...,Education,256463,3,19,Essential,Yes,gAAAAABnf1FDGe08lHq5NC2W9moILFDeej0mxcSJUtntLQ...,gAAAAABnf1FDaXeEt9kgiDlOLau2C6kAl3iiWFPT6rqSjX...,gAAAAABnf1FDcraCyVgpUOYDq4_1wMcBLvNJD7FvuBpDWc...,XXXXXX1466
297,शीला वर्मा,33,Odisha,Kendrapara,Small Business Owner,253688,25982,67633,Mutual Funds,Low,...,Agriculture Improvement,471260,10,17,Discretionary,No,gAAAAABnf1FDau49qt63YyqRor58tunQPigqCcDtLrir6K...,gAAAAABnf1FD5Au3zaw1lsv-mblpEVQxY_Ct3ob-YNvxVf...,gAAAAABnf1FDTpEhoGHPwTYvJKzLu56b6d58vsv_bomodE...,XXXXXX5233
298,माधवी गुहा,42,Jharkhand,Sheopur,Shopkeeper,191426,29750,428037,NaN,Low,...,Education,119921,5,15,Discretionary,Yes,gAAAAABnf1FDalwKOCkQclBvPCm0Y80ZW-0lI_MujmxMWi...,gAAAAABnf1FD9pNV_c-mhSJt1mxoPK0P9uxELFd5omyiJF...,gAAAAABnf1FDDdmV_FUB1wgM9SQ_6MjFguvSeVS9RRSVHa...,XXXXXX2653


# Feature Engineering

## Calculate Disposable Income:
Disposable Income = Annual Income - (Monthly Expenses * 12)

In [13]:
df['Disposable Income'] = df['Annual Income'] - (df['Monthly Expenses'] * 12)

## Investment Readiness Score:
Combine factors such as Net Worth, Emergency Fund Status, Risk Tolerance, and Savings Behavior.

In [15]:
# Define ranges for Savings Behavior categories
def categorize_savings_behavior(value):
    if 10 <= value <= 11:
        return 'Very Low'
    elif 12 <= value <= 13:
        return 'Low'
    elif 14 <= value <= 16:
        return 'Moderate'
    elif 17 <= value <= 19:
        return 'Good'
    elif 20 <= value <= 22:
        return 'Very Good'
    elif 23 <= value <= 25:
        return 'Excellent'
    elif 26 <= value <= 28:
        return 'Exceptional'
    elif 29 <= value <= 30:
        return 'Outstanding'
    else:
        return 'Unknown'  # Handle unexpected values

# Apply the categorization to the Savings Behavior column
df['Savings Behavior Category'] = df['Savings Behavior'].apply(categorize_savings_behavior)

In [16]:
def calculate_investment_readiness(row):
    # Assign weights to factors (customizable)
    net_worth_weight = 0.4
    emergency_fund_weight = 0.2
    risk_tolerance_weight = 0.2
    savings_behavior_weight = 0.2

    # Normalize numerical factors (e.g., Net Worth, Risk Tolerance)
    normalized_net_worth = row['Net Worth'] / df['Net Worth'].max()
    normalized_risk_tolerance = row['Risk Tolerance'] / df['Risk Tolerance'].max()

    # Encode Emergency Fund Status (Yes = 1, No = 0)
    emergency_fund_score = 1 if row['Emergency Fund Status'].lower() == 'yes' else 0

    # Savings Behavior numeric value
    savings_behavior_score = row['Savings Behavior'] / 30  # Normalize to [0, 1]

    # Weighted sum to calculate the Investment Readiness Score
    investment_score = (
        (normalized_net_worth * net_worth_weight) +
        (emergency_fund_score * emergency_fund_weight) +
        (normalized_risk_tolerance * risk_tolerance_weight) +
        (savings_behavior_score * savings_behavior_weight)
    )

    return round(investment_score * 100, 2)  # Scale score to [0, 100]

df['Investment Readiness Score'] = df.apply(calculate_investment_readiness, axis=1)

display(df)

TypeError: unsupported operand type(s) for /: 'str' and 'str'

In [ ]:
unique_values = df['Savings Behavior'].unique()
print(unique_values)